In [1]:
import os
os.chdir('/kaggle/input/myfunctions')
from functions import *
from myutils import *

In [2]:
!ls

functions.py  mytfutils.py  myutils.py


In [3]:
import pandas as pd
import numpy as np
import pickle
from functions import *
os.chdir('/kaggle/input/mydatasets2')
from sklearn.model_selection import train_test_split
import xgboost as xgb
from tqdm import tqdm

In [4]:
help(preprocess_data_via_close_values)

Help on function preprocess_data_via_close_values in module functions:

preprocess_data_via_close_values(pickle_file_path, time_steps=20, min_change=1e-05, shuffle=True, split=True, test_size=0.2, random_state=42)



In [5]:
num_features = 4 # Number of features in the data (Example: Low, High, etc)

time_steps = 20 # Time steps to consider
num_bagging_splits = 10
df_path = '/kaggle/input/mydatasets2/training_set1.pkl'
data, labels = preprocess_data_via_close_values(df_path, time_steps=20, shuffle=True, split=False)

Columns of each dataframe Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')
Shape of each array in the list (2181, 20)
Label Classes ['decrease' 'increase' 'no big change']
Data available for each label - increase, decrease and no big change 2107545 2132975 121480
Combined array shape -  (4362000, 20)
Combined labels shape -  (4362000,)
Shuffled data shape -  (4362000, 20)


In [6]:
data.shape

(4362000, 20)

In [7]:
labels.shape

(4362000,)

In [8]:
test_size=0.2
random_state=42
x_train, x_valid, y_train, y_valid = train_test_split(data, labels, test_size=test_size, random_state=random_state)

In [10]:
num_bagging_splits = 50

In [11]:
models_dict = {}
params = {'objective':'multi:softmax', 'eval_metric':'merror', 'num_class':'3', 'eta':'0.1', 'max_depth':'5'}
num_boost_rounds = 100
scaling = 2.5
for i in tqdm(range(num_bagging_splits)):
    indices = np.arange(0, len(x_train))
    np.random.shuffle(indices)
    slice_val = int(len(indices)//scaling)
    indices_selected = indices[:slice_val]
    x_tr = x_train[indices_selected]
    y_tr = y_train[indices_selected]
    xg_train = xgb.DMatrix(x_tr, label=y_tr)
    models_dict[i] = xgb.train(params, xg_train, num_boost_round=num_boost_rounds)

100%|██████████| 50/50 [29:21<00:00, 35.23s/it]


In [17]:
def xg_multi_predict(x = x_valid, y = y_valid, models_dict = models_dict):
    xg_valid = xgb.DMatrix(x, label=y)
    y_pred = np.zeros_like(y, dtype=np.float32)
    # y_pred = models_dict[0].predict(xg_valid)
    for xg_model in models_dict.values():
        y_pred += xg_model.predict(xg_valid)
    y_pred = y_pred/len(models_dict)
    return y_pred
y_pred = xg_multi_predict(x = x_valid, y = y_valid, models_dict = models_dict)
y_pred = np.round(y_pred)

In [18]:
accu, prec, recall, specificity, F1score = metric_calculations(y_pred, y_valid, set_="Validation set")

For Validation set
  Accuracy: 40.84364970197157,
  Precision: 49.765742625769064,
  Recall: 43.73107844044075,
  Specificity: 36.70334452910134,
OBOBOB  F1score: 46.55365967733038


In [15]:
df_path_test = "/kaggle/input/mydatasets2/training_set11.pkl"
x_test, y_test = preprocess_data_via_close_values(df_path_test, time_steps=20, shuffle=True, split=False)

Columns of each dataframe Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')
Shape of each array in the list (868, 20)
Label Classes ['decrease' 'increase' 'no big change']
Data available for each label - increase, decrease and no big change 1036659 1034361 52976
Combined array shape -  (2123996, 20)
Combined labels shape -  (2123996,)
Shuffled data shape -  (2123996, 20)


In [19]:
y_pred_test = xg_multi_predict(x = x_test, y = y_test, models_dict = models_dict)
accu1, prec1, recall1, specificity1, F1score1 = metric_calculations(y_pred_test, y_test, set_="Test set")

For Test set
  Accuracy: 6.786170972073394,
  Precision: 49.950527449655944,
  Recall: 3.3150008485846794,
  Specificity: 54.419552495742685,
OBOBOB  F1score: 6.217380965630303


In [20]:
output_file_path = "/kaggle/working/models_xg_bagging.pkl"
write_file_to_pickle(output_file_path, models_dict)